In [ ]:
# Dependencias del entrenamiento del modelo
!pip install torch torchvision matplotlib numpy pillow torchsummary tensorboard

# Dependencias de la API
!pip install zipfile36 flask flask-cors pyngrok torch torchvision

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import numpy as np
import os
from PIL import Image
import math
from torch.utils.tensorboard import SummaryWriter
import datetime
from torchsummary import summary
from torchvision.models.resnet import BasicBlock
import shutil
import zipfile


In [ ]:
print(torch.__version__)
print(torch.cuda.is_available())

In [ ]:
# Configuración del dispositivo (GPU si está disponible)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
# Verifica si existen los directorios
need_extract = False
for folder in ['datasets', 'examples']:
    if not os.path.exists(folder):
        need_extract = True
        break

if need_extract:
    zip_path = './folders.zip'  
    if os.path.exists(zip_path):
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall('.')
        print("¡Carpetas extraídas!")
    else:
        print("No se encontró el archivo folders.zip.")
else:
    print("Los directorios 'datasets' y 'examples' ya existen.")

In [ ]:
# Elimina la carpeta si existe
for folder in ['datasets/train/.ipynb_checkpoints', 'datasets/val/.ipynb_checkpoints']:
    if os.path.exists(folder):
        shutil.rmtree(folder)

In [ ]:
# Definir las transformaciones para los conjuntos de datos de entrenamiento y validación
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}


In [ ]:
# Definir las rutas a los directorios de datos
data_dir = 'datasets/train'
val_dir = 'datasets/val'
# Define el porcentaje de datos a utilizar (por ejemplo, 50%)
portion_train = 1
portion_val = 1

# Cargar los conjuntos de datos de entrenamiento y validación
full_train_dataset = datasets.ImageFolder(data_dir, data_transforms['train'])
full_val_dataset = datasets.ImageFolder(val_dir, data_transforms['val'])

# Calcular el número de muestras a usar según el porcentaje
num_train_samples = int(len(full_train_dataset) * portion_train)
num_val_samples = int(len(full_val_dataset) * portion_val)

# Crear un subset de los datos
train_indices = torch.randperm(len(full_train_dataset)).tolist()[:num_train_samples]
val_indices = torch.randperm(len(full_val_dataset)).tolist()[:num_val_samples]

image_datasets = {
    'train': torch.utils.data.Subset(full_train_dataset, train_indices),
    'val': torch.utils.data.Subset(full_val_dataset, val_indices)
}

# Crear DataLoaders
dataloaders = {
    'train': DataLoader(image_datasets['train'], batch_size=32, shuffle=True, num_workers=4),
    'val': DataLoader(image_datasets['val'], batch_size=32, shuffle=False, num_workers=4)
}

# Obtener las clases
class_names = full_train_dataset.classes

# Obtener el tamaño de los conjuntos de datos
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

In [ ]:
print(class_names)

In [ ]:
# Cargar un modelo pre-entrenado (ResNet18) y ajustar las últimas capas
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(class_names))
model = model.to(device)

In [ ]:
# Definir la función de pérdida y el optimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [ ]:
# Función para entrenar el modelo
def train_model(model, criterion, optimizer, num_epochs=5):
    best_model_wts = model.state_dict()
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch + 1}/{num_epochs}')
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
            

            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = model.state_dict()

        print()

    model.load_state_dict(best_model_wts)
    return model

In [ ]:
# Entrenar el modelo
model = train_model(model, criterion, optimizer, num_epochs=5)
# Guardar el modelo entrenado
torch.save(model.state_dict(), 'model.pth')

In [ ]:
# Función para predecir una imagen
def predict_image(image_path):
    model.eval()
    image = Image.open(image_path)
    image = data_transforms['val'](image).unsqueeze(0)
    image = image.to(device)

    with torch.no_grad():
        outputs = model(image)
        _, preds = torch.max(outputs, 1)
    
    return class_names[preds[0]]

In [ ]:

# Ruta a la carpeta que contiene las imágenes
folder_path = 'examples'

# Obtener una lista de todas las imágenes en la carpeta
image_files = [f for f in os.listdir(folder_path) if f.endswith(('.jpg', '.jpeg', '.png'))]

# Número de imágenes
num_images = len(image_files)

# Calcular el número de filas y columnas para el mosaico
cols = 3  # Puedes ajustar esto para tener más o menos columnas
rows = math.ceil(num_images / cols)

# Crear una figura con subplots
fig, axes = plt.subplots(rows, cols, figsize=(15, 5 * rows))

# Aplanar los ejes para iterar fácilmente si hay más de una fila
axes = axes.flatten()

# Iterar sobre las imágenes y mostrarlas en el mosaico
for i, filename in enumerate(image_files):
    image_path = os.path.join(folder_path, filename)
    
    # Hacer la predicción
    prediction = predict_image(image_path)
    
    # Abrir la imagen
    image = Image.open(image_path)
    
    # Mostrar la imagen junto con la predicción
    axes[i].imshow(image)
    axes[i].set_title(f'Predicción: {prediction}')
    axes[i].axis('off')

# Eliminar los ejes sobrantes si hay menos imágenes que subplots
for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()

In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS
from pyngrok import ngrok
import torch
from torchvision import models, transforms
from PIL import Image
import io

# https://dashboard.ngrok.com/get-started/your-authtoken
ngrok.set_auth_token("ngrok_token")

app = Flask(__name__)
CORS(app)

num_classes = len(class_names)
model = models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(model.fc.in_features, num_classes)
model.load_state_dict(torch.load('model.pth', map_location='cpu'))
model.eval()

data_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

@app.route('/predict', methods=['POST'])
def predict():
    if 'file' not in request.files:
        return jsonify({'error': 'No file provided'}), 400
    file = request.files['file']
    image = Image.open(io.BytesIO(file.read())).convert('RGB')
    image = data_transforms(image).unsqueeze(0)
    with torch.no_grad():
        outputs = model(image)
        _, preds = torch.max(outputs, 1)
    prediction = class_names[preds[0].item()]
    return jsonify({'prediction': prediction})

public_url = ngrok.connect(5000)
print(" * ngrok URL:", public_url)
app.run(port=5000)